![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.3.prepare_CoNLL_from_annotations_for_NER.ipynb)

# 1.3. Prepare CoNLL file from annotations for NER

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp

In [2]:
import json
import os

import pandas as pd
from tqdm import tqdm
from collections import Counter

import sparknlp
from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from sparknlp.training import CoNLL

import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


spark = sparknlp.start()

print ("Spark NLP Version :", sparknlp.version())

spark

Spark NLP Version : 5.5.0


## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

### Entity File
This dataframe should include at least five  below columns in order:
*   ['text_id','begin','end','chunk','entity']

In [3]:
import pandas as pd

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ChemProt/chemprot_train_entities.csv

train_entities_df = pd.read_csv('chemprot_train_entities.csv')
train_entities_df= train_entities_df[["text_id", "begin", "end", "chunk", "entity"]]
train_entities_df.head()

,text_id,begin,end,chunk,entity
0,11319232,242,250,acyl-CoAs,CHEMICAL
1,11319232,1193,1200,triacsin,CHEMICAL
2,11319232,1441,1447,sucrose,CHEMICAL
3,11319232,1637,1651,triacylglycerol,CHEMICAL
4,11319232,1702,1710,acyl-CoAs,CHEMICAL


### Text File
This dataframe should include at least two below columns in order:
*   `['text_id','text']`

In [4]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ChemProt/chemprot_train_text.csv
train_text_df = pd.read_csv('chemprot_train_text.csv')
train_text_df.head()

,text_id,text
0,16357751,Selective costimulation modulators: a novel ap...
1,14967461,Emerging role of epidermal growth factor recep...
2,23468099,Effects of chronic social defeat stress on beh...
3,23293962,Hepatocyte growth factor activator inhibitor t...
4,7678677,Alprenolol and bromoacetylalprenololmenthane a...


In [ ]:
def make_conll(text:pd.DataFrame, entity:pd.DataFrame,
               save_tag:bool=None,
               save_conll:bool=None,
               verbose:bool=None,
               begin_deviation:int=0,
               end_deviation:int=0 )->str:

    df_text = text.iloc[:,[0,1]]
    df_entity = entity.iloc[:,[0,1,2,3,4]]
    df_text.columns = ['text_id','text']
    df_entity.columns = ['text_id','begin','end','chunk','entity']
    entity_list = list(df_entity.entity.unique())


    ########--------------1.tag transformation function------------########

    def transform_text(text, entities, verbose=None):

        tag_list=[]
        for entity in entities.iterrows():

            begin = entity[1][1] + begin_deviation
            end = entity[1][2] + end_deviation
            chunk = entity[1][3]
            tag = entity[1][4]
            text = text[:end] + f' </END_NER:{tag}> ' + text[end:]
            text = text[:begin] + f' <START_NER:{tag}> ' + text[begin:]
            tag_list.append(tag)

        sum_of_added_entity = Counter(tag_list)
        sum_of_entity = Counter(entities['entity'].values)

        if verbose:
            print(f'Processed text id   : {entities.text_id.values[:1]}')
            print(f'Original Entities   : {sum_of_entity}\nAdded Entities      : {sum_of_added_entity}')
            print(f'Number Equality     : {sum_of_added_entity == sum_of_entity}')
            print("=="*40)

        if not sum_of_entity == sum_of_added_entity:
            print("There is a problem in text id:")
            print(entities.text_id.values[0])
            raise Exception("Check this text!")

        return text


    ######---------------2.apply_transform_text function ----------------#######

    def apply_tag_ner(df_text, df_entity, save=None, verbose=None):

        for text_id in tqdm(df_text.text_id):
            text  = df_text.loc[df_text['text_id']==text_id]['text'].values[0]
            entities  = df_entity.loc[(df_entity['text_id']==text_id)].sort_values(by='begin',ascending=False)

            df_text.loc[df_text['text_id']==text_id, 'text'] = transform_text(text, entities, verbose=verbose)

        if save:
            df_text.to_csv("text_with_ner_tag.csv", index=False, encoding='utf8')

        return df_text


    ##########----------------3.RUNNING TAG FUNCTION---------------#############

    print("Text tagging starting. Applying entities to whole text...\n")
    df = apply_tag_ner(df_text, df_entity, save=save_tag, verbose=verbose)


    ###########---------------4.Spark Pipeline-----------------------###########

    def spark_pipeline(df):
        spark_df = spark.createDataFrame(df)

        documentAssembler = DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")\
            .setCleanupMode("shrink")

        sentenceDetector = SentenceDetector()\
            .setInputCols(['document'])\
            .setOutputCol('sentences')\
            .setExplodeSentences(True)

        tokenizer = Tokenizer() \
            .setInputCols(["sentences"]) \
            .setOutputCol("token")

        nlpPipeline = Pipeline(stages=[documentAssembler, sentenceDetector, tokenizer ])

        empty_df = spark.createDataFrame([['']]).toDF("text")
        pipelineModel = nlpPipeline.fit(empty_df)

        result = pipelineModel.transform(spark_df.select(['text']))


        return result.select('token.result').toPandas()
    print("\n\nSpark pipeline is running...")
    df_final = spark_pipeline(df)


    #########--------------5.CoNLL Function--------------------#############

    def build_conll(df_final, tag_list, save=None):

        header = "-DOCSTART- -X- -X- O\n\n"
        conll_text = ""
        chunks = []
        tag_list = tag_list
        tag = 'O'      # token tag
        ct = 'B'       # chunk tag part B or I

        for sentence_tokens in tqdm(df_final.result[:]):
            for token in sentence_tokens:
                if token.startswith("<START_NER:"):
                    tag = token.split(':')[1][:-1]
                    if tag not in tag_list:
                        tag = 'O'
                        conll_text += f'{token} NN NN {tag}\n'

                    continue

                if token.startswith("</END_NER:") and tag != 'O':
                    for i, chunk in enumerate(chunks):
                        ct = 'B' if i == 0 else 'I'
                        conll_text += f'{chunk} NNP NNP {ct}-{tag}\n'

                    chunks=[]
                    tag='O'
                    continue

                if tag != 'O':
                    chunks.append(token)
                    continue

                if tag == 'O':
                    conll_text += f'{token} NN NN {tag}\n'
                    continue

            conll_text += '\n'

        if save:
            with open("conll2003_text_file.conll", "w+", encoding='utf8') as f:
                f.write(header)
                f.write(conll_text)

        print("\nDONE!")
        return conll_text


    ########----------------6.RUNNING CONLL FUNCTION--------------------########

    print("Conll file is being created...\n")
    return build_conll(df_final, tag_list=entity_list, save=save_conll)


## Running the Create CoNLL Function

In [ ]:
# if you want tagged text or conll file saved in the current directory: just make default 'save_tag' or 'save_conll' parameters True.
conll_text = make_conll(train_text_df, train_entities_df, save_conll=True)

Text tagging starting. Applying entities to whole text...



  0%|          | 0/1020 [00:00<?, ?it/s]<ipython-input-5-41e6c86c8703>:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  begin = entity[1][1] + begin_deviation
<ipython-input-5-41e6c86c8703>:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end = entity[1][2] + end_deviation
<ipython-input-5-41e6c86c8703>:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chunk = entity[1][3]
<ipython-input-5-41e6c86c8703>:25: FutureWarning: Series._



Spark pipeline is running...
Conll file is being created...



100%|██████████| 10835/10835 [00:00<00:00, 88941.50it/s]


DONE!


In [ ]:
# Checking conll string
print(conll_text[:532])

Selective NN NN O
costimulation NN NN O
modulators NN NN O
: NN NN O
a NN NN O
novel NN NN O
approach NN NN O
for NN NN O
the NN NN O
treatment NN NN O
of NN NN O
rheumatoid NN NN O
arthritis NN NN O
. NN NN O

T NN NN O
cells NN NN O
have NN NN O
a NN NN O
central NN NN O
role NN NN O
in NN NN O
the NN NN O
orchestration NN NN O
of NN NN O
the NN NN O
immune NN NN O
pathways NN NN O
that NN NN O
contribute NN NN O
to NN NN O
the NN NN O
inflammation NN NN O
and NN NN O
joint NN NN O
destruction NN NN O
characteristic NN NN O



### Saving CoNLL File

In [ ]:
with open("conll2003_text_file.conll", "w+", encoding='utf8') as f:
    f.write("-DOCSTART- -X- -X- O\n\n")
    f.write(conll_text)

# Reading CoNLL File by Using CoNLL Reader

In [ ]:
data = CoNLL().readDataset(spark, "/content/conll2003_text_file.conll")
data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Selective costimu...|[{document, 0, 96...|[{document, 0, 96...|[{token, 0, 8, Se...|[{pos, 0, 8, NN, ...|[{named_entity, 0...|
|T cells have a ce...|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 0, T,...|[{pos, 0, 0, NN, ...|[{named_entity, 0...|
|The requirement f...|[{document, 0, 22...|[{document, 0, 22...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|This approach is ...|[{document, 0, 78...|[{document, 0, 78...|[{token, 0, 3, Th...|[{pos, 0, 3, NN, ...|[{named_entity, 0...|
|it targets events...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 1, it...|[{pos, 0, 1, NN, ..

# Alternative Method

This method may take some time depending on the size of your data.

In [5]:
train_entities_df.head()

,text_id,begin,end,chunk,entity
0,11319232,242,250,acyl-CoAs,CHEMICAL
1,11319232,1193,1200,triacsin,CHEMICAL
2,11319232,1441,1447,sucrose,CHEMICAL
3,11319232,1637,1651,triacylglycerol,CHEMICAL
4,11319232,1702,1710,acyl-CoAs,CHEMICAL


In [6]:
train_text_df.head()

,text_id,text
0,16357751,Selective costimulation modulators: a novel ap...
1,14967461,Emerging role of epidermal growth factor recep...
2,23468099,Effects of chronic social defeat stress on beh...
3,23293962,Hepatocyte growth factor activator inhibitor t...
4,7678677,Alprenolol and bromoacetylalprenololmenthane a...


In [7]:
spark_df = spark.createDataFrame(train_text_df)
spark_df.show()

+--------+--------------------+
| text_id|                text|
+--------+--------------------+
|16357751|Selective costimu...|
|14967461|Emerging role of ...|
|23468099|Effects of chroni...|
|23293962|Hepatocyte growth...|
| 7678677|Alprenolol and br...|
|16554356|Covalent linkage ...|
|16789740|Discovery and opt...|
| 7526860|Keratinocyte grow...|
|17199504|Configuration of ...|
|16437532|Cholinesterase in...|
|11716850|The effects of mi...|
|23487168|A systematic comp...|
|23019138|Diuretic effects ...|
|12065695|Nonsteroidal anti...|
|23214423|Anisotropy of che...|
|16840830|Two novel mutatio...|
|15736931|Mutation of argin...|
|23414802|Discovery of nove...|
|23578689|Conformationally ...|
|15908512|Beta1 adrenergic ...|
+--------+--------------------+
only showing top 20 rows



In [8]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer])

result = nlpPipeline.fit(spark_df).transform(spark_df)

In [9]:
result.select("text_id",F.explode(F.arrays_zip(result.token.result,
                                            result.token.begin,
                                            result.token.end,
                                            result.token.metadata)).alias("cols"))\
                  .select("text_id",F.expr("cols['0']").alias("token"),
                          F.expr("cols['3']['sentence']").alias("sentence_id"),
                          F.expr("cols['1']").alias("token_begin"),
                          F.expr("cols['2']").alias("token_end")).show(50, truncate=100)

+--------+--------------+-----------+-----------+---------+
| text_id|         token|sentence_id|token_begin|token_end|
+--------+--------------+-----------+-----------+---------+
|16357751|     Selective|          0|          0|        8|
|16357751| costimulation|          0|         10|       22|
|16357751|    modulators|          0|         24|       33|
|16357751|             :|          0|         34|       34|
|16357751|             a|          0|         36|       36|
|16357751|         novel|          0|         38|       42|
|16357751|      approach|          0|         44|       51|
|16357751|           for|          0|         53|       55|
|16357751|           the|          0|         57|       59|
|16357751|     treatment|          0|         61|       69|
|16357751|            of|          0|         71|       72|
|16357751|    rheumatoid|          0|         74|       83|
|16357751|     arthritis|          0|         85|       93|
|16357751|             .|          0|   

In [10]:
test_token_df=result.select("text_id",F.explode(F.arrays_zip(result.token.result,
                                            result.token.begin,
                                            result.token.end,
                                            result.token.metadata)).alias("cols"))\
                  .select("text_id",F.expr("cols['0']").alias("token"),
                          F.expr("cols['3']['sentence']").alias("sentence_id"),
                          F.expr("cols['1']").alias("begin"),
                          F.expr("cols['2']").alias("end")).toPandas()

In [11]:
new_token_df = test_token_df.copy()
new_token_df["entity"] = "O"

inner_label=False
for i, token_row in test_token_df.iterrows():
    text_id = token_row.text_id
    token_begin = token_row.begin
    token_end = token_row.end


    for j, entity_row in train_entities_df[train_entities_df.text_id == text_id].iterrows():
        entity_begin = entity_row.begin
        entity_end = entity_row.end


        if (entity_begin<= token_begin) and (token_end<= entity_end):
            if new_token_df.loc[i-1,"entity"] == new_token_df.loc[i,"entity"] :
                new_token_df.loc[i,"entity"] = "B-"+(entity_row.entity)
            else:
                new_token_df.loc[i,"entity"] = "I-"+(entity_row.entity)

In [12]:
new_token_df.to_csv("train_new_token_df.csv",index=False)

In [13]:
conll_text = "-DOCSTART- -X- -0- O\n\n"
sentence_index = 0
document_index = 0


for i, row in new_token_df.iterrows():
    if document_index == row.text_id:

        if sentence_index == row.sentence_id:
            conll_text += f"{row.token} NN NN {row.entity}\n"

        else:
            conll_text += "\n"
            sentence_index = row.sentence_id
            conll_text += f"{row.token} NN NN {row.entity}\n"

    else:
        document_index = row.text_id
        conll_text += f"\n-DOCSTART- -X- -{row.text_id}- O\n"
        conll_text += "\n"
        conll_text += f"{row.token} NN NN {row.entity}\n"
        sentence_index = row.sentence_id
# print(conll_text)

In [14]:
print(conll_text[1000:])

Output hidden; open in https://colab.research.google.com to view.

In [15]:
with open("train.conll", "w+", encoding='utf8') as f:
    f.write("-DOCSTART- -X- -X- O\n\n")
    f.write(conll_text)

In [16]:
data = CoNLL().readDataset(spark, "train.conll")
data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Selective costimu...|[{document, 0, 96...|[{document, 0, 96...|[{token, 0, 8, Se...|[{pos, 0, 8, NN, ...|[{named_entity, 0...|
|T cells have a ce...|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 0, T,...|[{pos, 0, 0, NN, ...|[{named_entity, 0...|
|The requirement f...|[{document, 0, 22...|[{document, 0, 22...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|This approach is ...|[{document, 0, 78...|[{document, 0, 78...|[{token, 0, 3, Th...|[{pos, 0, 3, NN, ...|[{named_entity, 0...|
|it targets events...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 1, it...|[{pos, 0, 1, NN, ..